In [ ]:
!pip install tensorflow keras pillow pyttsx3

In [ ]:
!pip install -q kaggle

In [ ]:
!pip install Opendatasets

In [ ]:
import opendatasets as od
import pandas
data_dir = od.download(
    "https://www.kaggle.com/datasets/adityajn105/flickr8k",
force='True',
    data_dir ='flickr8k_data'  # Specify a different extraction directory
)

In [ ]:
!pip install gTTS

In [ ]:
import tensorflow as tf
import numpy as np
import json
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50, MobileNet
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
from tensorflow.keras.applications.mobilenet import preprocess_input as preprocess_input_mobilenet
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, Add
from gtts import gTTS
import pyttsx3
import IPython.display as ipd
from pycocotools.coco import COCO
import requests
from io import BytesIO
from PIL import Image





In [ ]:
data_dir = "/content/flickr8k_data/flickr8k"

In [ ]:
import pathlib
data_dir= pathlib.Path(data_dir)  #to add dataset path in directory
data_dir

In [ ]:
list(data_dir.glob('*/*.jpg'))[:5]

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
# Load libraries
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np
import os
import cv2
import keras
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model
# np_utils has been moved to tensorflow.keras.utils
from tensorflow.keras.utils import to_categorical
import random
from keras.preprocessing import image, sequence
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet50, decode_predictions as decode_resnet50
from tensorflow.keras.applications.mobilenet import preprocess_input as preprocess_mobilenet, decode_predictions as decode_mobilenet
from tensorflow.keras.applications import ResNet50, MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Concatenate, Dense
from tensorflow.keras.models import Model
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
from gtts import gTTS
import IPython.display as ipd


In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
# Load the BLIP processor and model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [ ]:

def load_and_preprocess_image(img_path, target_size):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [ ]:
# Function to add prefix to layer names
def add_prefix_to_layers(model, prefix):
    for layer in model.layers:
        layer._name = prefix + layer.name

# Load Pretrained Models
resnet_model = ResNet50(weights='imagenet', include_top=False)
add_prefix_to_layers(resnet_model, "resnet_")

mobilenet_model = MobileNet(weights='imagenet', include_top=False)
add_prefix_to_layers(mobilenet_model, "mobilenet_")


In [ ]:
# Load Pretrained Models
resnet_model = ResNet50(weights='imagenet', include_top=False)
mobilenet_model = MobileNet(weights='imagenet', include_top=False)


In [ ]:
img_path = "/content/flickr8k_data/flickr8k/Images/1007320043_627395c3d8.jpg"  # Removed extra '.jpg' from the file name
img_resnet50 = preprocess_resnet50(load_and_preprocess_image(img_path, target_size=(224, 224)))
img_mobilenet = preprocess_mobilenet(load_and_preprocess_image(img_path, target_size=(224, 224)))

In [ ]:
def create_hybrid_model():
    resnet_input = Input(shape=(224, 224, 3), name='resnet_input')
    mobilenet_input = Input(shape=(224, 224, 3), name='mobilenet_input')

    resnet_output = resnet_model(resnet_input)
    mobilenet_output = mobilenet_model(mobilenet_input)

    x_resnet50 = GlobalAveragePooling2D()(resnet_output)
    x_mobilenet = GlobalAveragePooling2D()(mobilenet_output)

    concatenated = Concatenate()([x_resnet50, x_mobilenet])
    predictions = Dense(1000, activation='softmax')(concatenated)

    model = Model(inputs=[resnet_input, mobilenet_input], outputs=predictions)
    return model

hybrid_model = create_hybrid_model()



In [ ]:
def load_and_preprocess_image(img_path, target_size):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

img_path = '/content/flickr8k_data/flickr8k/Images/1001773457_577c3a7d70.jpg'  # replace with your image path
img_resnet50 = preprocess_resnet50(load_and_preprocess_image(img_path, target_size=(224, 224)))
img_mobilenet = preprocess_mobilenet(load_and_preprocess_image(img_path, target_size=(224, 224)))


In [ ]:
# Make prediction
preds = hybrid_model.predict([img_resnet50, img_mobilenet])
predicted_class = np.argmax(preds[0])

# Load ImageNet labels
!wget https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json
import json
with open('imagenet_class_index.json', 'r') as f:
    class_dict = json.load(f)
class_labels = {int(key): value[1] for key, value in class_dict.items()}

predicted_label = class_labels[predicted_class]
print("Predicted label:", predicted_label)


In [ ]:
# Extract Features
features = hybrid_model.predict([img_resnet50, img_mobilenet])

In [ ]:
# Ensure transformers library is installed
!pip install -q transformers

from transformers import GPT2Tokenizer, TFGPT2LMHeadModel

In [ ]:
# Load the tokenizer and the model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
captioning_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Prepare input for the model
input_ids = tokenizer.encode("Features: ", return_tensors="tf")

In [ ]:
# Generate caption
output = captioning_model.generate(input_ids, max_length=50, num_return_sequences=1)
caption = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated caption:", caption)

In [ ]:
# Load and preprocess image
img_path = "/content/flickr8k_data/flickr8k/Images/1001773457_577c3a7d70.jpg"  # Local file path
img = Image.open(img_path) # Open the image directly
inputs = processor(img, return_tensors="pt")

In [ ]:
!pip install -q transformers
from transformers import BlipForConditionalGeneration

# Assuming 'model' was originally a PretrainedResNetModel, replace it with:
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Now, the 'generate' method should be available
output = model.generate(**inputs)
caption = processor.decode(output[0], skip_special_tokens=True)

print("Generated caption:", caption)

In [ ]:
!pip install googletrans==4.0.0-rc1 gtts

In [ ]:
from googletrans import Translator
from gtts import gTTS
import IPython.display as ipd

# Prompt user for language code
def get_language_code():
    # List of language codes for common languages
    languages = {
        'English': 'en',
        'Spanish': 'es',
        'French': 'fr',
        'German': 'de',
        'Chinese': 'zh',
        'Japanese': 'ja',
        'Korean': 'ko',
        'Italian': 'it',
        'Hindi': 'hi',
        'Tamil': 'ta',
        'Bengali': 'bn',
        'Telugu': 'te',
        'Punjabi': 'pa',
        'Marathi': 'mr',
        'Russian': 'ru',
        'Arabic': 'ar',
        'Portuguese': 'pt',
        'Dutch': 'nl',
        'Swedish': 'sv',
        'Norwegian': 'no',
        'Greek': 'el',
        'Turkish': 'tr',
        'Persian': 'fa',
    }

    print("Available languages:")
    for lang_name, lang_code in languages.items():
        print(f"{lang_name}: {lang_code}")

    lang_choice = input("Enter the language code (e.g., 'en', 'es'): ").strip()
    return lang_choice if lang_choice in languages.values() else 'en'  # Default to 'en' if not found

# Translate text to the desired language
def translate_text(text, dest_lang):
    translator = Translator()
    translated = translator.translate(text, dest=dest_lang)
    return translated.text

# Convert text to speech
def text_to_speech(text, lang='en'):
    tts = gTTS(text=text, lang=lang)
    tts.save('caption.mp3')
    return 'caption.mp3'

# Example usage
language_code = get_language_code()  # Get language code from user
translated_caption = translate_text(caption, language_code)  # Translate caption
print(f"Translated Caption: {translated_caption}")
voice_output = text_to_speech(translated_caption, lang=language_code)  # Generate speech

# Play the audio
ipd.display(ipd.Audio(voice_output))


In [ ]:
# Bagging - wrong output
from collections import Counter

def bagging_ensemble(caption_file_path):
    """
    Performs bagging ensemble on captions from a file.

    Args:
        caption_file_path: Path to the file containing captions, one per line.

    Returns:
        The most frequent caption.
    """
    with open(caption_file_path, 'r') as f:
        captions = [line.strip() for line in f]

    return Counter(captions).most_common(1)[0][0]

# Example usage:
caption_file_path = "/content/flickr8k_data/flickr8k/captions.txt" # Replace with your actual file path
result = bagging_ensemble(caption_file_path)
print(result)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import torchvision.transforms as transforms

# Placeholder - Replace these with your actual data
x_data = torch.randn(100, 3, 224, 224)  # Example: 100 samples with 3 channels (RGB) and 224x224 dimensions
y_data = torch.randint(0, 2, (100, 1)).float()  # Example: 100 binary target values

# Create a dataset (Do not normalize yet)
dataset = TensorDataset(x_data, y_data)

# Split the data into train and test sets (80-20 split)
train_size = int(0.8 * len(x_data))
test_size = len(x_data) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

from torchvision import models

class PretrainedResNetModel(nn.Module):
    def __init__(self):
        super(PretrainedResNetModel, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        x = self.sigmoid(x)
        return x

# Initialize model, loss function, and optimizer
model = PretrainedResNetModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Lower learning rate for fine-tuning

from torchvision.transforms import ToTensor, Normalize, Compose, Resize
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, x, y, transform=None):
        self.x = x
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        # Ensure x is in the shape [C, H, W] before applying transforms
        if x.dim() == 2:  # If it's a 2D tensor, assume it's grayscale and add a channel dimension
            x = x.unsqueeze(0)
        if self.transform:
            x = self.transform(x)
        return x, y

# Define the transformations - Remove ToTensor as the data is already in Tensor format
transform = Compose([
    Resize((224, 224)),  # Resize images to 224x224
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet mean and std
])

# Create the DataLoader
train_loader = DataLoader(CustomDataset(train_dataset.dataset.tensors[0], train_dataset.dataset.tensors[1], transform=transform), batch_size=32, shuffle=True)
test_loader = DataLoader(CustomDataset(test_dataset.dataset.tensors[0], test_dataset.dataset.tensors[1], transform=transform), batch_size=32, shuffle=False)
        # Continue with the rest of your training loop logic here

In [ ]:
# Define the model with Pretrained ResNet
class PretrainedResNetModel(nn.Module):
    def __init__(self):
        super(PretrainedResNetModel, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        x = self.sigmoid(x)
        return x

# Initialize model, loss function, and optimizer
model = PretrainedResNetModel().to(torch.float32)  # Ensure model parameters are in float32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss().to(device)  # Ensure loss function is on the correct device
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [ ]:
# Convert data tensors to float32
x_data = torch.tensor(x_data, dtype=torch.float32)
y_data = torch.tensor(y_data, dtype=torch.float32)


In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomCrop(224, padding=4),  # Add random cropping
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), # Add color jittering
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

     # Apply this transform when creating your datasets and dataloaders.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import torchvision.transforms as transforms
from torchvision import models

# ... (Your existing code for data loading, dataset creation, etc.)

class PretrainedResNetModel(nn.Module):
    def __init__(self):
        super(PretrainedResNetModel, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.5)  # Add dropout layer here

    def forward(self, x):
        x = self.resnet(x)
        x = self.dropout(x)  # Apply dropout before sigmoid
        x = self.sigmoid(x)
        return x

# Use weight decay in your optimizer:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

In [ ]:
from sklearn.metrics import accuracy_score
epochs = 20
for epoch in range(epochs):
     model.train()
     epoch_loss = 0
     y_true_train = []  # To store true labels for the training set
     y_pred_train = []  # To store predicted labels for the training set

     for batch_x, batch_y in train_loader:
         batch_x, batch_y = batch_x.to(device).to(torch.float32), batch_y.to(device).to(torch.float32)

         optimizer.zero_grad()
         outputs = model(batch_x)
         loss = criterion(outputs, batch_y)
         loss.backward()
         optimizer.step()

         epoch_loss += loss.item()

         # Get predictions and true labels for accuracy calculation
         _, predicted = torch.max(outputs, 1)
         y_true_train.extend(batch_y.cpu().numpy())
         y_pred_train.extend(predicted.cpu().numpy())

     epoch_loss /= len(train_loader)
     train_accuracy = accuracy_score(y_true_train, y_pred_train)  # Calculate accuracy
     print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

Epoch 1/20, Loss: 0.9300, Training Accuracy: 0.5100
Epoch 2/20, Loss: 0.5065, Training Accuracy: 0.5100
Epoch 3/20, Loss: 0.4473, Training Accuracy: 0.5100
Epoch 4/20, Loss: 0.4571, Training Accuracy: 0.5100
Epoch 5/20, Loss: 0.4788, Training Accuracy: 0.5100
Epoch 6/20, Loss: 0.2632, Training Accuracy: 0.5100
Epoch 7/20, Loss: 0.0835, Training Accuracy: 0.5100
Epoch 8/20, Loss: 0.0810, Training Accuracy: 0.5100
Epoch 9/20, Loss: 0.0141, Training Accuracy: 0.5100
Epoch 10/20, Loss: 0.0283, Training Accuracy: 0.5100
Epoch 11/20, Loss: 0.0091, Training Accuracy: 0.5100
Epoch 12/20, Loss: 0.0040, Training Accuracy: 0.5100
Epoch 13/20, Loss: 0.0033, Training Accuracy: 0.5100
Epoch 14/20, Loss: 0.1146, Training Accuracy: 0.5100
Epoch 15/20, Loss: 0.1630, Training Accuracy: 0.5100
Epoch 16/20, Loss: 0.2566, Training Accuracy: 0.5100
Epoch 17/20, Loss: 0.1033, Training Accuracy: 0.5100


In [ ]:
# Evaluate the model
# Evaluate the model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # Import necessary metrics

model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device).to(torch.float32), batch_y.to(device).to(torch.float32)
        outputs = model(batch_x).cpu().numpy()
        y_true.extend(batch_y.cpu().numpy())
        y_pred.extend(outputs)

# Convert predictions to binary (0 or 1)
y_pred = (np.array(y_pred) > 0.5).astype(int)
y_true = np.array(y_true).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Plot metrics
import matplotlib.pyplot as plt

metrics = {
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
}

plt.figure(figsize=(10, 6))
plt.bar(metrics.keys(), metrics.values(), color=['b', 'g', 'r', 'c'])
plt.title('Evaluation Metrics')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.grid(True)
plt.show()


In [ ]:


def calculate_mcc(tp, tn, fp, fn):
    """
    Calculate the Matthews Correlation Coefficient (MCC).

    Parameters:
    tp (int): True Positives
    tn (int): True Negatives
    fp (int): False Positives
    fn (int): False Negatives

    Returns:
    float: The MCC score
    """
    # Calculate the MCC
    numerator = (tp * tn) - (fp * fn)
    denominator = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    if denominator == 0:
        return 0.0  # Return 0 if the denominator is zero to avoid division by zero

    return numerator / denominator

# Calculate tp, tn, fp, fn from y_true and y_pred
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

mcc_score = calculate_mcc(tp, tn, fp, fn)
print(f'MCC Score: {mcc_score:.4f}')

In [ ]:
!pip install matplotlib seaborn

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Assuming y_true and y_pred are available as NumPy arrays
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix using Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
y_pred_proba = model(batch_x).detach().cpu().numpy()  # Store probabilities

In [ ]:
# Evaluate the model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

model.eval()
y_true = []
y_pred = []
y_pred_proba = []  # Store probabilities for all batches

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device).to(torch.float32), batch_y.to(device).to(torch.float32)
        outputs = model(batch_x)
        y_true.extend(batch_y.cpu().numpy())
        y_pred.extend(outputs.cpu().numpy() > 0.5)  # Directly store binary predictions
        y_pred_proba.extend(outputs.cpu().numpy())  # Store probabilities

# Convert lists to numpy arrays
y_true = np.array(y_true).astype(int)
y_pred = np.array(y_pred).astype(int)
y_pred_proba = np.array(y_pred_proba).squeeze()  # Squeeze to get a 1D array of probabilities


# Calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
roc_auc = auc(fpr, tpr)
print(f'ROC AUC: {roc_auc:.4f}')

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import numpy as np

variance = np.var(y_pred_proba)
print("Variance of model predictions:", variance)

In [ ]:
def count_layers(model):
    """Counts the number of layers in a PyTorch model."""
    layer_count = 0
    for name, module in model.named_modules():
        if isinstance(module, (torch.nn.Linear, torch.nn.Conv2d, torch.nn.BatchNorm2d, torch.nn.ReLU, torch.nn.MaxPool2d)):  # Add other layer types as needed
            layer_count += 1
    return layer_count

num_layers = count_layers(model)
print(f"The model has {num_layers} layers.")

In [ ]:
# Assuming 'model' is your PyTorch model, 'test_data', and 'test_labels' are your test data
import torch

# Define your loss function
criterion = torch.nn.CrossEntropyLoss()

# ... (Your PyTorch evaluation loop)
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    outputs = model(x_data)
    # Assuming 'predicted_label' should be a tensor of class indices
    # Replace this with the actual way to get the correct class indices for your data
    predicted_label = torch.tensor([0] * len(outputs))  # Example: all samples belong to class 0
    loss = criterion(outputs, predicted_label)
    _, predicted = torch.max(outputs, 1)
    top1_accuracy = (predicted == predicted_label).sum().item() / len(predicted_label)

print("Test Loss:",loss.item())
print("Top-1 Accuracy:", top1_accuracy)